In [9]:
#import
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [10]:
#define
SESSION_PATH = './sessions'
WELLNESS_PATH = './wellness'
PICKLE_PATH = './pickle/'
format = '%Y-%m-%d'
lista_colonne_utili_analisi_esercizi = ['splitName',
                                        'workload',
                                        'perceivedWorkload',
                                        'metabolicWorkload',
                                        'mechanicalWorkload',
                                        'kinematicWorkload',
                                        'cardioWorkload']

In [11]:
def from_str_to_date(stringa):
    timestr = stringa.split(' ')[0]
    time = datetime.strptime(timestr, format)
    return time

#MEAN AVERAGE DEVIATION
mad = lambda x: np.fabs(x - x.mean()).mean()

In [25]:
#importo csv esercizi
df_exercises = pd.read_pickle(PICKLE_PATH + 'df_exercises.pickle')
df_exercises

,splitName,workload,perceivedWorkload,metabolicWorkload,mechanicalWorkload,kinematicWorkload,cardioWorkload
0,Run-2,NaN,0.0,0.0,0.0,0.0,0
1,Run-3,NaN,0.0,0.0,0.0,0.0,0
2,Passing-drill 3,NaN,0.0,0.0,0.0,0.0,0
3,Session,2.3,77.2,3.7,25.3,0.0,0
4,Passing-drill 2,0.5,0.0,2.5,14.3,0.0,0
5,Run-and passing dr,0.9,0.0,3.7,25.3,0.0,0
6,Run-1,NaN,0.0,0.0,0.0,0.0,0
7,Passing-drill 1,NaN,0.0,0.0,0.0,0.0,0
8,Passing-drill 1,0.5,0.0,0.0,10.5,0.0,0
9,Run-2,NaN,0.0,0.0,0.0,0.0,0


In [26]:
#lista a degli esercizi
list_exercises = list(df_exercises['splitName'].unique())
#list_exercises = list_exercises.remove('Session')
list_exercises

['Run-2',
 'Run-3',
 'Passing-drill 3',
 'Session',
 'Passing-drill 2',
 'Run-and passing dr',
 'Run-1',
 'Passing-drill 1',
 'First-half game',
 'Square-5vs5 20x20m',
 'Theme-game 10vs10 ',
 'Second-half game',
 'warm-up and mov sk',
 'Possession-10vs10 ',
 'Warm-up',
 'Yo-yo-IRT L2',
 'Game-half pitch',
 'Rehab',
 'Aerobic-work',
 'First-intermittent',
 'First-possession',
 'Second-possession',
 'Game-66m',
 'Second-intermitten',
 'Passing-drill squa',
 'Shooting',
 'Shifting-and ball ',
 '11vs1',
 'Functional-trainin',
 '4vs4-plus 2 or 4vs',
 '4vs4-plus 2 and 4v',
 'Defensive-tactic',
 'Intermittent-run',
 'Second-half-game',
 'First-aerobic work',
 'Second-aerobic wor',
 'Aerobic-work plus ',
 'Warm-up plus mov. ',
 '1vs1-2vs2 4vs4 4vs',
 'Game-11vs11',
 'First-half',
 'Second-half',
 'Warm-up pre match',
 'Set-pieces',
 'Hand-ball',
 'Preparation-to the',
 'Aerobic-circuit',
 'Build-up',
 'Game-position 4vs3',
 'Boxes',
 'Second-game 9vs9',
 'Possession-game 6v',
 'First-game 9vs9

In [47]:
df_mean = pd.DataFrame()

#drop della colonna cardio
df_exercises_temp = df_exercises.drop(['cardioWorkload'],axis=1)

for exercise in list_exercises:
    
    #prendo il singolo esercizio (droppo colonna del nome)
    df_single_exercise = df_exercises_temp[df_exercises['splitName'].isin([exercise])].drop(['splitName'], axis=1)
    
    #trasformo gli 0 in nan
    df_single_exercise = df_single_exercise.replace(0.0, np.nan)
    
    temp_df = pd.DataFrame(columns = ['splitName'] + 
                           [x+'_mean' for x in df_single_exercise.columns.values] + 
                           [x+'_std' for x in df_single_exercise.columns.values])
    
    record = [exercise]
    
    #creo colonna di media e varianza per ogni categoria
    for column in df_single_exercise.columns.values:
        record += [df_single_exercise[column].mean()]
    
    for column in df_single_exercise.columns.values:    
        record += [df_single_exercise[column].std()]

    temp_df.loc[0] = record
    
    df_mean = df_mean.append(temp_df) 
    
#     print('NOME ESERCIZIO: ',exercise)
#     print(df_single_exercise)
#     print('MEDIE ESERCIZIO:')
#     print(df_single_exercise.mean())
#     print('MAD ESERCIZIO:')
#     print(df_single_exercise.drop(['splitName'],axis=1).apply(mad))
#     print('MAD percentuale:')
#     print(df_single_exercise.drop(['splitName'],axis=1).apply(mad) / df_single_exercise.mean() * 100)
#     print('\n')

df_mean = df_mean.reset_index(drop= True)
df_mean

,splitName,workload_mean,perceivedWorkload_mean,metabolicWorkload_mean,mechanicalWorkload_mean,kinematicWorkload_mean,workload_std,perceivedWorkload_std,metabolicWorkload_std,mechanicalWorkload_std,kinematicWorkload_std
0,Run-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Run-3,0.873684,NaN,18.163158,4.966667,NaN,0.196757,NaN,3.145757,2.136196,NaN
2,Passing-drill 3,0.250000,NaN,1.300000,5.500000,NaN,0.213809,NaN,NaN,4.193249,NaN
3,Session,1.811813,36.015065,33.479778,35.582333,38.206513,0.877562,17.432237,23.613265,18.065103,21.764191
4,Passing-drill 2,0.414286,NaN,2.780000,13.820000,NaN,0.247848,NaN,1.023230,4.434749,NaN
5,Run-and passing dr,1.100000,NaN,5.416667,25.936364,NaN,0.382099,NaN,1.800463,7.329021,NaN
6,Run-1,0.726316,NaN,14.531579,6.000000,22.150000,0.182093,NaN,2.364095,3.541186,8.491761
7,Passing-drill 1,0.475000,NaN,2.600000,11.228571,NaN,0.205287,NaN,1.131371,3.744202,NaN
8,First-half game,0.437500,NaN,7.700000,9.470833,8.476923,0.113492,NaN,2.105686,3.201016,3.188300
9,Square-5vs5 20x20m,0.509091,NaN,2.675000,11.836364,NaN,0.197254,NaN,0.942956,5.155438,NaN


In [48]:
df_mean.to_json('Esercizi.txt',orient='records')